# SVD Classifier

In [12]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt 
import time
import random
#!pip install ProgressBar
from progressbar import ProgressBar
import math
#!{sys.executable} -m pip install ProgressBar
#!pip install ProgressBar
from random import randrange
from copy import copy, deepcopy

import glob
import json

#svd stuff
from scipy.linalg import svd 
from numpy.linalg import pinv 


In [13]:
#load the titles and wheater the book is alive or dead into two seperate arrays
def loadResult():
    test_labels = []
    test_values = []
    with open('table-of-contents.json', encoding="utf8") as f:
        data = json.load(f)
        for books in data:
            for book in data[books]:
                test_labels.append(book['title'])
                test_values.append(book['dead'])
                
    return test_labels, test_values

#get the data from the data output file
def loadData():
    final_data = []
    with open('dataOutput2.json', encoding="utf8") as f:
        data = json.load(f)
        for index in data: 
            final_data.append(data[index]['chapData'])
    return final_data

In [14]:

                

title_labels, dead_labels = loadResult()
train = loadData()
print(len(train))
## Load the training set
train_data= np.array(train[:15705])
train_labels = np.array(dead_labels[:15705])
train_titles = np.array(title_labels[:15705])


## Load the testing set

test_data= np.array(train[-1000:]) 
test_labels = np.array(dead_labels[-1000:])
test_titles = np.array(title_labels[-1000:])


## Print out their dimensions
print("Training dataset dimensions: ", np.shape(train_data))
print("Number of training labels: ", len(train_labels))
print("Testing dataset dimensions: ", np.shape(test_data))
print("Number of testing labels: ", len(test_labels))



16705
Training dataset dimensions:  (15705, 71)
Number of training labels:  15705
Testing dataset dimensions:  (1000, 71)
Number of testing labels:  1000


For the next two items we are attempting to recreating the element. A should equal B


calculate pseudo inverse manually and compare to the built in version

In [15]:
def getPseudo (curr):
    
    # calculate svd
    U, s, VT = svd(curr)
    # reciprocals of s
    d = 1.0 / s
    # create m x n D matrix
    D = np.zeros(curr.shape)
    # populate D with n x n diagonal matrix
    D[:curr.shape[1], :curr.shape[1]] = np.diag(d)
    # calculate pseudoinverse
    new = VT.T.dot(D.T).dot(U.T)
    
    return new




In [17]:
A = train_data

# calculate pseudoinverse
print('Puilt in PINV function')
B = pinv(A)
print(B)
print('Our method')
B=getPseudo(A)
print(B)



Puilt in PINV function
[[ 1.41665154e-06 -7.87624385e-05 -1.50063288e-05 ... -8.81508881e-05
  -2.43381414e-04 -1.57186015e-04]
 [-3.11887480e-05  4.14462675e-05 -2.39004888e-04 ... -1.38122724e-04
  -1.99278900e-05 -3.38553861e-04]
 [-2.30330965e-05 -3.87923977e-05 -8.42779723e-05 ... -1.93302046e-04
  -1.29650184e-05  1.44900618e-03]
 ...
 [-3.53724560e-07  4.86100109e-07  1.05452792e-06 ...  6.99338964e-07
   5.91041688e-07 -3.66002797e-06]
 [ 4.49952314e-07 -4.67608076e-07 -1.13338360e-07 ... -6.02837625e-07
  -5.32801834e-07  3.21050947e-06]
 [-3.71539162e-09 -5.57172465e-08  5.07890139e-07 ... -1.54623509e-07
  -2.89767954e-08 -2.49413822e-07]]
Our method
[[ 1.41665154e-06 -7.87624385e-05 -1.50063288e-05 ... -8.81508881e-05
  -2.43381414e-04 -1.57186015e-04]
 [-3.11883520e-05  4.14462206e-05 -2.39004961e-04 ... -1.38122710e-04
  -1.99278888e-05 -3.38553853e-04]
 [-2.30322454e-05 -3.87925141e-05 -8.42781799e-05 ... -1.93302006e-04
  -1.29647602e-05  1.44900626e-03]
 ...
 [-3.53724

In [18]:
def getError(matrix, data, labels):

    true = 0
    
    len(data)
    print(len(matrix))
    
    for i in range(len(data)):
        x = data[i]
        
        predict = np.sum(x.dot(matrix))

        #find the error
        if (labels[i] == False) and (predict<1):
            true +=1
            #print(predict, ': ', labels[i])
        elif (labels[i] == True) and (predict>1):
            true+=1
       
       

        #these are  the results from the svd classification
        #print(predict, end = ": ")
        #print(train_labels[i])

        #calculate the error

    
    return 1-(true/len(data))

In [19]:
print('error rate: ', getError(B, train_data, train_labels))
print('test rate ', getError(B, test_data, test_labels))

71
error rate:  0.3444126074498567
71
test rate  0.32699999999999996


Here we are going to create a new psuedo inverse matrix. We will exclude all of the genres in the dataset to see if we can get a lower initial error rate.  This matrix will only be a 5*5 matrix.

In [33]:
##try to get the psudo inverse when excluding the genre from the list

print('New matrix')
newA = A[:,-5:]
print(newA)
print('New Pseudo')
newB = getPseudo(newA)
print(newB)
print(len(newB))

print('error rate: ', getError(newB, train_data[:,-5:], train_labels))
print('test rate ', getError(newB, test_data[:,-5:], test_labels))

New matrix
[[ 849    2    0    1   36]
 [ 336    0    0    1    7]
 [ 417   70   20   25  689]
 ...
 [ 144    1    1    0    5]
 [ 460    6    2    0    1]
 [2975  827  187  255  232]]
New Pseudo
[[ 2.30179326e-08  9.91475252e-09 -1.34144187e-09 ...  3.27235073e-09
   1.03180175e-08  1.56591788e-08]
 [-2.23284894e-07 -9.99779258e-08 -9.79382441e-08 ... -3.38490848e-08
  -8.69787770e-08  6.93662401e-07]
 [-3.06234988e-07 -1.34135158e-07 -2.48061348e-07 ... -2.21253227e-08
  -1.11250122e-07 -3.53011980e-06]
 [ 7.23994847e-07  3.36342813e-07  7.46779978e-08 ...  8.22741266e-08
   2.64910168e-07  2.85604162e-06]
 [ 1.78131789e-08  7.97238553e-10  5.41029713e-07 ...  2.25907157e-09
  -5.47407311e-09 -6.19685001e-08]]
5
5
error rate:  0.8406876790830946
5
test rate  0.841


# Compute the derivative


We will now compute the derivative of the regression cost function: 
$$L_D(w) = \frac{1}{n}\sum_{i=1}^n (y_i-w\cdot x_i)^2,$$
where $x_i\in \mathrm{R}^d$ is the input feature of dimension $d$, $y_i\in\mathrm{R}$ is the output response, and $w\in\mathrm{R}^d$ is the regression weights.

**Task P3:** Complete the function 'weight_derivative' to calculate the derivative of the cost function with respect to regression weights $w$, i.e., $\frac{\partial}{\partial w}L_D(w)$. Note that this should be a $d$ dimensional vector. Also copy the output of the code for the test example to the solution file.

In [21]:
def weight_derivative(weights, feature_matrix, labels):
    # Input:
    # weights: weight vector w, a numpy vector of dimension d
    # feature_matrix: numpy array of size n by d, where n is the number of data points, and d is the feature dimension
    # labels: true labels y, a numpy vector of dimension d
    # Output:
    # Derivative of the regression cost function with respect to the weight w, a numpy array of dimension d
        
    ## STUDENT: Start of code ###
    
    test_predictions = np.dot(weights,feature_matrix)
    
    
    
    errors = test_predictions - labels
    
    derivative = np.dot(feature_matrix,np.transpose(errors))
    derivative = derivative#/(np.sum(derivative))
    return(derivative)
    # End of code ###

In [22]:
# NOTE: copy the output to the solution file.



my_weights = np.zeros(len(train_data[0])) # this makes all the predictions 0
derivative = weight_derivative(my_weights, B, train_labels)

print (derivative)

[ 3.04014813e-03  8.46879732e-03 -1.75732122e-02 -4.83218283e-03
  3.31761432e-04  3.86677752e-03 -1.78805059e-02 -8.96592632e-03
  1.26142539e-02  9.93006212e-04 -6.79400073e-03 -2.14297097e-04
  3.06249437e-03 -2.12486455e-03 -3.32189037e-03 -9.25336117e-05
 -5.08664039e-03  5.34560516e+06  1.46648453e-02  8.18348156e-03
 -8.81335130e-03  1.77514288e-02  1.07599593e-02  2.09512774e-03
  7.10672677e-03 -2.64451790e-03 -1.96833512e-02  1.21160566e-02
 -5.79651531e-03  7.60269106e-03 -9.53161241e-03 -5.25935632e-04
 -2.48322560e-02 -1.18838144e-02 -4.99126297e-03  7.36531047e-03
  1.69510026e-02 -1.88833970e-02  1.03962200e-02 -9.69900785e-03
 -1.72892240e-03 -7.62163761e-03 -1.56968660e-02  4.91079001e-03
  2.18878440e-02  7.87371709e-03 -1.52332115e-02 -5.49918155e-03
  2.76562561e-04  2.55078104e-02 -7.38967766e-03  1.57961186e+08
  1.91096773e-05 -2.73740747e-03  3.23087238e-04 -3.96663658e-04
 -2.46739209e-03 -1.31894691e-01  7.02742986e-03 -6.61882453e-01
 -2.21172657e-01  6.31141

Here, we will write a function to perform gradient descent algorithm on the lineare regression cost. Given an initial point, we will update the current weights by moving in the negative gradient direction to minimize the cost function. Thus, in each iteration we obtain the updated weight $w_{t+1}$ from the current iterate $w_t$ as follows:
$$w_{t+1} = w_t - h\frac{\partial}{\partial w}L_D(w_t),$$
where $h$ is the 'step_size' that is the amount by which we move in the negative gradient direction. 

We stop when we are sufficiently close to the optimum (where gradient is the zero vector) by checking the condition with respect to the magnitude (length) of the gradient vector:
$$\|\frac{\partial}{\partial w}L_D(w_t)\|_2\leq \epsilon,$$
where $\epsilon$ is the 'tolerance' parameter.

**Task P4:** Complete the code section to perform the gradient decent in the function `regression_gradient_descent`. Copy the code to the solution file.

In [43]:
def regression_gradient_descent(feature_matrix, labels, initial_weights, step_size, tolerance):
    # Gradient descent algorithm for linear regression problem    
    
    # Input:
    # feature_matrix: numpy array of size n by d, where n is the number of data points, and d is the feature dimension
    # labels: true labels y, a numpy vector of dimension d
    # initial_weights: initial weight vector to start with, a numpy vector of dimension d
    # step_size: step size of update
    # tolerance: tolerace epsilon for stopping condition
    # Output:
    # Weights obtained after convergence
    
    converged = False 
    weights = np.array(initial_weights) # current iterate
    i = 0
    while not converged:
        # Start of code: your impelementation of what the gradient descent algorithm does in every iteration
        # Refer back to the update rule listed above: update the weight
        i += 1
        derivative = weight_derivative(weights, feature_matrix, labels)
        #print(derivative)
        weights -= (step_size * derivative)
        
        # Compute the gradient magnitude:
        
        gradient_magnitude = np.sqrt(np.sum(derivative**2))
        
        # Check the stopping condition to decide whether you want to stop the iterations
        #print ("grad mag :", gradient_magnitude)
        #print ("tolerance:", tolerance)
        if gradient_magnitude > tolerance:
            converged = True
        
        # End of code
        
        print ("Iteration: ",i,"gradient_magnitude: ", gradient_magnitude) # for us to check about convergence
        x = np.dot(np.transpose(weights),B)
        print('error rate: ', getError(x, train_data[:2000], train_labels[:2000]))

        #print('error rate: ', getError(x, train_data, train_labels))
        
    return weights

In [44]:
#STUDENT: Specify the initial_weights, step_size, and tolerance
simple_feature_matrix = B
output = train_labels


temp_weights = [np.ones(len(train_data[0]))]*(len(train_data[0]))# this makes all the predictions 0 #np.array([0.]*2*len(simple_feature_matrix))
temp_weights=np.array(temp_weights)
initial_weights = temp_weights.reshape(len(simple_feature_matrix),len(train_data[0]))

step_size = 0.5
tolerance = 2
# end of code

# Use the regression_gradient_descent function to calculate the gradient decent and store it in the variable 'final_weights'
final_weights = regression_gradient_descent(simple_feature_matrix, output, initial_weights, step_size, tolerance)

# end of code
print ("Here are the final weights after convergence:")

print (final_weights)


Iteration:  1 gradient_magnitude:  3.8999877318054246e+17
71
error rate:  0.11950000000000005
Here are the final weights after convergence:
[[ 0.99852215  0.99852215  0.99852215 ...  0.99852215  0.99852215
   0.99852215]
 [ 4.14749385  4.14749385  4.14749385 ...  4.14749385  4.14749385
   4.14749385]
 [20.3919445  20.3919445  20.3919445  ... 20.3919445  20.3919445
  20.3919445 ]
 ...
 [ 1.00123865  1.00123865  1.00123865 ...  1.00123865  1.00123865
   1.00123865]
 [ 1.00103201  1.00103201  1.00103201 ...  1.00103201  1.00103201
   1.00103201]
 [ 1.00044657  1.00044657  1.00044657 ...  1.00044657  1.00044657
   1.00044657]]


In [38]:
#print(len(B[0]))
#print(len(final_weights[0]))

x = np.dot(np.transpose(final_weights),B)

#print(len(x))
print('error rate: ', getError(x, train_data, train_labels))
print('test rate: ', getError(x, test_data, test_labels))

15705
71
71
71
error rate:  0.1217446673034066
71
test rate:  0.10899999999999999
